In [65]:
Packages <- c("dplyr",  "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer", "viridis")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
#source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_grid_search.R")
source("~/oceanography/scripts/LL_biophys.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R")
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained
source("~/oceanography/scripts/format_genetic_parentage_matrix.R")
source("~/oceanography/scripts/format_genetic_kernel_parentage_matrix.R")
source("~/oceanography/scripts/format_biophys_normalized_matrix.R")
source("~/oceanography/scripts/format_biophys_parentage_matrix.R")
set.seed(7)
#source("~/oceanography/scripts/PredictedProportions.R")

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
kernel2012_14 <- fread(file="~/oceanography/empirical_data/genetics/GenKernelsForROMSComp2012-14.csv")

#read in the necessary input data for likelihood functions
load(file= "~/oceanography/script_output/SurveyData/for_likelihood_functions/2022-01-20_AddDestTables.Rdata")
load(file= "~/oceanography/script_output/ROMSDataTables/2021-11-04_SeededParticleInfo.Rdata")
load(file= "~/oceanography/script_output/SurveyData/for_likelihood_functions/2021-11-04_InputSurveyData.Rdata")



In [69]:
#what if for the genetic kernel matrices we allow the two magbangons
AllRecruitsGen <- fread("~/parentage/kernel_fitting/1340_loci/final_results/tables/AllFishObsWithPar.csv")
#head(AllRecruitsGen)
MonsoonRecSampPar <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/RecruitsByMonsoon2012-14ForROMSComp.csv")
#head(MonsoonRecSampPar)

#make one data table to rule them all....

AllRecruitsGenWithMonsoonNSMag <- left_join(AllRecruitsGen, MonsoonRecSampPar[, .(fish_indiv, year, monsoon=season)], by=c("fish_indiv", "year"))#[
   # offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"][year %in% c(2012, 2013, 2014)]
SurveyData <- fread("~/oceanography/script_output/SurveyData/SurveyDataNSMag.csv")


genetic_kernel_matrices <- suppressMessages(format_genetic_kernel_parentage_matrix(AllRecruitsGenWithMonsoonNSMag, SurveyData))

fwrite(genetic_kernel_matrices$KernelGenMat2012_4, file="~/oceanography/script_output/SurveyData/2022-01-20_KernelParentageMatrix2012-14ForROMSCompNSMag.csv")
fwrite(genetic_kernel_matrices$KernelGenMat2012, file="~/oceanography/script_output/SurveyData/2022-01-20_KernelParentageMatrix2012ForROMSCompNSMag.csv")
fwrite(genetic_kernel_matrices$KernelGenMat2013, file="~/oceanography/script_output/SurveyData/2022-01-20_KernelParentageMatrix2013ForROMSCompNSMag.csv")
fwrite(genetic_kernel_matrices$KernelGenMat2014, file="~/oceanography/script_output/SurveyData/2022-01-20_KernelParentageMatrix2014ForROMSCompNSMag.csv")
fwrite(genetic_kernel_matrices$KernelGenMatNEM, file="~/oceanography/script_output/SurveyData/2022-01-20_KernelParentageMatrixNEMForROMSCompNSMag.csv")
fwrite(genetic_kernel_matrices$KernelGenMatSWM, file="~/oceanography/script_output/SurveyData/2022-01-20_KernelParentageMatrixSWMForROMSCompNSMag.csv")


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


In [70]:
sum(genetic_kernel_matrices$KernelGenMat2012)

[1] 63

In [46]:
genetic_kernel_matrices$KernelGenMat2012

,Cabatoan,N.Magbangon,Palanas,Poroc Rose,Poroc San Flower,S.Magbangon,San Agustin,Sitio Lonas,Visca,Wangag
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,1
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,1,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,1,0,0,0,0,0


In [37]:
dcast(full_join(AllRecruitsGen[year==2012 & matched_offs=="Y", .N, by=c("par_site", "offs_site")], unique(AddDestGen[year==2012, -c("year", "monsoon")]))[, N :=0][order(par_site, offs_site)], par_site ~ offs_site, value.var="N", fun.aggregate = )#[, -"par_site"]  #[is.na(N), N :=0]
#t(as.matrix(full_join(AllRecruitsGen[year==2012 & matched_offs=="N", .N, by=site][order(site)], AddDestGen[year==2012, .(site=offs_site)])[, -"site"])))


Joining, by = c("par_site", "offs_site")


par_site,Cabatoan,N.Magbangon,Palanas,Poroc Rose,Poroc San Flower,S.Magbangon,San Agustin,Sitio Lonas,Visca,Wangag
Cabatoan,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
N.Magbangon,NA,0,NA,NA,NA,NA,NA,NA,NA,NA
Palanas,NA,NA,0,NA,NA,NA,NA,NA,NA,0
Poroc Rose,NA,NA,NA,0,NA,NA,NA,NA,NA,NA
Poroc San Flower,NA,NA,NA,NA,0,NA,NA,NA,NA,NA
S.Magbangon,NA,NA,NA,NA,0,0,NA,NA,NA,NA
San Agustin,NA,NA,NA,NA,NA,NA,0,NA,NA,NA
Sitio Lonas,NA,NA,NA,NA,NA,NA,NA,0,NA,NA
Visca,NA,NA,NA,NA,NA,NA,NA,NA,0,NA
Wangag,NA,NA,NA,NA,0,NA,NA,NA,NA,0


In [57]:
AddDestGen

offs_site,par_site,year,monsoon
Cabatoan,Cabatoan,2012,NEM
N.Magbangon,N.Magbangon,2012,NEM
Palanas,Palanas,2012,NEM
Poroc Rose,Poroc Rose,2012,NEM
Poroc San Flower,Poroc San Flower,2012,NEM
S.Magbangon,S.Magbangon,2012,NEM
San Agustin,San Agustin,2012,NEM
Sitio Lonas,Sitio Lonas,2012,NEM
Visca,Visca,2012,NEM
Wangag,Wangag,2012,NEM


In [63]:
KernelGenMat2012 <- rbind(as.matrix(dcast(full_join(AllRecruitsGenWithMonsoonNSMag[year==2012 & matched_offs=="Y", .N, by=c("offs_site", "par_site")], AddDestGen[year==2012, -c("year", "monsoon")])[order(offs_site, par_site)], par_site ~ offs_site, value.var="N")[, -"par_site"]),  #[is.na(N), N :=0]
t(as.matrix(full_join(AllRecruitsGenWithMonsoonNSMag[year==2012 & matched_offs=="N", .N, by=site], unique(AddDestGen[year==2012, .(site=offs_site)]))[order(site)][, -"site"])))

KernelGenMat2012[is.na(KernelGenMat2012)] <- 0

Warning message in `[.data.table`(AddDestGen, year == 2012, -c("year", "monsoon")):
“column(s) not removed because not found: monsoon”Joining, by = c("offs_site", "par_site")
Joining, by = "site"


In [64]:
KernelGenMat2012

,Cabatoan,N.Magbangon,Palanas,Poroc Rose,Poroc San Flower,S.Magbangon,San Agustin,Sitio Lonas,Visca,Wangag
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,1
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,1,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,1,0,0,0,0,0


In [2]:
AllRecruitsGen <- fread("~/parentage/kernel_fitting/1340_loci/final_results/tables/AllFishObsWithPar.csv")
#head(AllRecruitsGen)
MonsoonRecSampPar <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/RecruitsByMonsoon2012-14ForROMSComp.csv")
#head(MonsoonRecSampPar)

#make one data table to rule them all....

AllRecruitsGenWithMonsoon <- left_join(AllRecruitsGen, MonsoonRecSampPar[, .(fish_indiv, year, monsoon=season)], by=c("fish_indiv", "year"))[
    offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"][year %in% c(2012, 2013, 2014)]

genetic_parentage_matrices <- suppressMessages(format_genetic_parentage_matrix(AllRecruitsGenWithMonsoon, SurveyData))
#fwrite(genetic_parentage_matrices$GenMat2012_4, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2012-14ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2012, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2012ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2013, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2013ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2014, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2014ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMatNEM, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrixNEM2012-14ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMatSWM, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrixSWM2012-14ForROMSComp.csv")


genetic_kernel_matrices <- suppressMessages(format_genetic_kernel_parentage_matrix(AllRecruitsGenWithMonsoon, SurveyData))

#fwrite(genetic_kernel_matrices$KernelGenMat2012_4, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012-14ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2012, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2013, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2013ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2014, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2014ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMatNEM, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixNEM2012-14ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMatSWM, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixSWM2012-14ForROMSComp.csv")


__read in the biophysical data and format for likelihood functions__

In [248]:
SimConn <- fread(file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTableCompleteMetaLongForm08DayPLD.csv")[year %in% c(2012, 2013, 2014) & dest != "CAI"][#filter out CAI as a destination for now, not very well spatially defined
   ,`:=`(daily_particles_rec=as.numeric(daily_particles_rec),daily_particles_released=as.numeric(daily_particles_released))][
    source == "Other" | source == "CAI", source:="unknown"][ #aggregate the CAI and other sites as "unknown for likelihood functions"
    , daily_particles_rec := sum(daily_particles_rec), by=c("date", "source", "dest")][
   , daily_particles_released :=sum(as.numeric(daily_particles_released)), by=c("date", "source")]
total_release_days

[1] 687

In [249]:

biophys_norm_matrices <- format_biophys_normalized_matrix(SimConn=SimConn, total_release_days=687, AddDestSim=AddDestSim)
#fwrite(biophys_norm_matrices$FullBiophysMatNorm, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2012-14ForROMSComp08DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2012, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2012ForROMSComp08DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2013, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2013ForROMSComp08DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2014, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2014ForROMSComp08DayPLD.csv")
#fwrite(biophys_norm_matrices$MonsoonBiophysMatNormNEM, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrixNEMForROMSComp08DayPLD.csv" )
#fwrite(biophys_norm_matrices$MonsoonBiophysMatNormSWM, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrixSWMForROMSComp08DayPLD.csv" )

biophys_parentage_matrices <- format_biophys_parentage_matrix(SimConn=SimConn, total_release_days=687, AddDestSim=AddDestSim)
#fwrite(biophys_parentage_matrices$FullBiophysMat, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2012-14ForROMSComp08DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2012, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2012ForROMSComp08DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2013, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2013ForROMSComp08DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2014, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2014ForROMSComp08DayPLD.csv")
#fwrite(biophys_parentage_matrices$MonsoonBiophysMatNEM, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrixNEMForROMSComp08DayPLD.csv")
#fwrite(biophys_parentage_matrices$MonsoonBiophysMatSWM, file="~/oceanography/script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrixSWMForROMSComp08DayPLD.csv")

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


In [95]:
head(SimConn)

particle_id,source,dest,year,monsoon,date,num_route_parentage_matches,num_release_days_seasonal,num_release_days_annual,dest_num_females,...,source_num_rec_sampled_annual,source_prop_anem_samp,sim_month,sim_day,sim_year,daily_particles_rec,daily_particles_released,dist_km,bearing,direction
P54642131,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA
P54642132,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA
P54642133,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA
P54642134,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA
P54642135,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA
P54642136,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA


In [232]:
sum(biophys_parentage_matrices$AnnualBiophysMat2012)+sum(biophys_parentage_matrices$AnnualBiophysMat2013)+sum(biophys_parentage_matrices$AnnualBiophysMat2014)
sum(biophys_parentage_matrices$FullBiophysMat)

[1] 955263

[1] 955263

# Get the best-fit kernels and profile likelihood confidence intervals in fit_data.ipynb


In [270]:
sampled_reefs_vec <- as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]) #***I think these index values need to match dimensions of parentage matrix- filter out sand flats and other unsampled sites***
pop_size_vec <- as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]) #vector of pop sizes for all reefs (a). This term is also used in parentage kernel fitting, but reef sizes are substituted as a proxy for pop size. This is should be bootstrapped to account for uncertainty.
prop_samp_vec <- as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)])#vector of proportion of habitat sampled for all reefs in time period t- I think that I should be coming up with a different value for this, but for now this will help me code the function

#bootstrap the particle AND parentage data and re-fit kernels and perform bode likelihood function for dispersal event matching
num_samples_sim <- nrow(SimConn) #because each row in SimConn is a particle the number of rows is N samples per iteration for bootstrapping

#make table to hold results
BootKernels <- data.table(data_source=character(), time_scale=character(), time_id=character(), k=numeric(), theta=numeric(),num_matches=numeric(), mdd=numeric(), med=numeric(), dist90=numeric(), log_like=numeric(), iteration=numeric())

BootLikelihood <- data.table(comparison=character(), time_scale=character(), log_like=numeric(), iteration=numeric())

pb <- txtProgressBar(min = 0, max = 1000, style = 3)
AllRecruitsGen <- fread("~/parentage/kernel_fitting/1340_loci/final_results/tables/AllFishObsWithPar.csv")
MonsoonRecSampPar <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/RecruitsByMonsoon2012-14ForROMSComp.csv")


 AllRecruitsGenWithMonsoon <- left_join(AllRecruitsGen, MonsoonRecSampPar[, .(fish_indiv, year, monsoon=season)], by=c("fish_indiv", "year"))[
        offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"][year %in% c(2012, 2013, 2014)]    
 #    , .SD[sample(.N, 394, replace=TRUE)]] #resample the recruits with replacement not by the same number at each site

TotalOffsBySiteYear <- AllRecruitsGenWithMonsoon[, .(num_gen=.N), by=c("year", "site")]

AllRecruitsGenWithMonsoon2 <- left_join(AllRecruitsGenWithMonsoon, TotalOffsBySiteYear, by=c("year", "site"))

######################################
#for kernels, use the matrices with NS magbangon seperated
TotalOffsBySiteYearNSMag <- AllRecruitsGenWithMonsoonNSMag[, .(num_gen=.N), by=c("year", "site")]
AllRecruitsGenWithMonsoon2NSMag <- left_join(AllRecruitsGenWithMonsoonNSMag, TotalOffsBySiteYearNSMag, by=c("year", "site"))
CentroidsNSMag <- fread(file="~/oceanography/empirical_data/site_centroids_SimTestNSMag.csv") 
CentroidsNSMag$site <- gsub("_full", "", CentroidsNSMag$site, fixed=TRUE)
CentroidsNSMag$site <- gsub("_ten_per_cover", "", CentroidsNSMag$site, fixed=TRUE)

CentroidsNSMag <- CentroidsNSMag %>%
    arrange(site)
### List of source locations
SitesSource <- CentroidsNSMag

### List of destination locations
SitesDest <- CentroidsNSMag

DistMatm <- distm(SitesSource[,c('lon','lat')], SitesSource[,c('lon','lat')], fun=distVincentyEllipsoid)
DistancesNSMag <- DistMatm*10^-3

AreaNSMag <- fread("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels_testNSMag.csv") %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6)# %>%
    #select(kmsq) #need to uncomment for functions to work
AreaNSMag$site <- gsub("_ten_per_cover", "", AreaNSMag$site, fixed=TRUE)
reef_sizesNSMag <- as.matrix(AreaNSMag$kmsq)

SiteIndexNSMag <- unique(CentroidsNSMag %>% arrange(site), by="site")[, index := .I] #add the row number as the unique site index, leave CAI in if fitting a kernel SurveyDataNSMag <- fread("~/oceanography/script_output/SurveyData/SurveyDataNSMag.csv")
######################################
for(i in 1:1000){
    
    #resample dispersal data, both genetics and simulation
    SimConnInt2 <- SimConn[, .SD[sample(.N, num_samples_sim, replace=TRUE)]] #sample the particle data
    #test case subsampling the particle data
    #SimConnInt2 <- SimConnSubSamp[!is.na(num_gen), .SD[sample(.N, num_gen, replace=TRUE)], by=c("year", "dest")] 

#make one data table to rule them all....

 #   AllRecruitsGenWithMonsoon <- left_join(AllRecruitsGen, MonsoonRecSampPar[, .(fish_indiv, year, monsoon=season)], by=c("fish_indiv", "year"))[
 #       offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"][year %in% c(2012, 2013, 2014)]    
 ##    , .SD[sample(.N, 394, replace=TRUE)]] #resample the recruits with replacement not by the same number at each site
 #  
    #TotalOffsBySiteYear <- AllRecruitsGenWithMonsoon[, .(num_gen=.N), by=c("year", "site")]
    
    #test case redoing with subsampling by site and year instead of total recruits sampled
    AllRecruitsGenWithMonsoonInt <- AllRecruitsGenWithMonsoon2[, .SD[sample(.N, num_gen, replace=TRUE)], by=c("year", "site")] #resample the recruits with replacement
    AllRecruitsGenWithMonsoonNSMagInt <- AllRecruitsGenWithMonsoon2NSMag[, .SD[sample(.N, num_gen, replace=TRUE)], by=c("year", "site")] #resample the recruits with replacement


    
        #format the matrices for kernel fitting and likelihood function
        genetic_parentage_matrices <- suppressMessages(format_genetic_parentage_matrix(AllRecruitsGenWithMonsoon=AllRecruitsGenWithMonsoonInt, SurveyData))
        genetic_kernel_matrices <- suppressMessages(format_genetic_kernel_parentage_matrix(AllRecruitsGenWithMonsoon=AllRecruitsGenWithMonsoonNSMagInt, SurveyData=SurveyDataNSMag))
        biophys_norm_matrices <- format_biophys_normalized_matrix(SimConn=SimConnInt2, total_release_days=687, AddDestSim = AddDestSim)
        biophys_parentage_matrices <- format_biophys_parentage_matrix(SimConn=SimConnInt2, total_release_days=687, AddDestSim = AddDestSim)



##fit the kernels
biophys_par_data2012 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2012), ncol=1, 1))
Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012, control=list(maxit=500)))

biophys_par_data2013 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2013), ncol=1, 1))
Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2013, control=list(maxit=500)))

biophys_par_data2014 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2014), ncol=1, 1))
Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2014, control=list(maxit=500)))

biophys_par_data2012_4 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$FullBiophysMat, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2014)))
Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012_4, control=list(maxit=500)))

biophys_par_dataNEM <- list(Distances=Distances, Assignments=biophys_parentage_matrices$MonsoonBiophysMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$MonsoonBiophysMatNEM), ncol=1, 1))
SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataNEM, control=list(maxit=500)))

biophys_par_dataSWM <- list(Distances=Distances, Assignments=biophys_parentage_matrices$MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$MonsoonBiophysMatSWM), ncol=1, 1))
SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataSWM, control=list(maxit=500)))

#genetic data with same reef metadata structure as model

par_data2012 <- list(Distances=DistancesNSMag, Assignments=genetic_kernel_matrices$KernelGenMat2012, Sampled_reefs=t(as.matrix(SiteIndexNSMag[site %in% SurveyDataNSMag[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizesNSMag, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2012 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012, control=list(maxit=500)))

par_data2013 <- list(Distances=DistancesNSMag, Assignments=genetic_kernel_matrices$KernelGenMat2013, Sampled_reefs=t(as.matrix(SiteIndexNSMag[site %in% SurveyDataNSMag[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizesNSMag, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2013 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2013, control=list(maxit=500)))

par_data2014 <- list(Distances=DistancesNSMag, Assignments=genetic_kernel_matrices$KernelGenMat2014, Sampled_reefs=t(as.matrix(SiteIndexNSMag[site %in% SurveyDataNSMag[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2014, control=list(maxit=500)))

par_data2012_4 <- list(Distances=DistancesNSMag, Assignments=genetic_kernel_matrices$KernelGenMat2012_4, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizesNSMag, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012_4, control=list(maxit=500)))

par_dataNEM <- list(Distances=DistancesNSMag, Assignments=genetic_kernel_matrices$KernelGenMatNEM, Sampled_reefs=t(as.matrix(SiteIndexNSMag[site %in% SurveyDataNSMag[year==2014  & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizesNSMag, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014   & prop_anem_samp >0, .(prop_anem_samp)]))
GenNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataNEM, control=list(maxit=500)))

par_dataSWM <- list(Distances=DistancesNSMag, Assignments=genetic_kernel_matrices$KernelGenMatSWM, Sampled_reefs=t(as.matrix(SiteIndexNSMag[site %in% SurveyDataNSMag[year==2014  & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizesNSMag, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014   & prop_anem_samp >0, .(prop_anem_samp)]))
GenSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataSWM, control=list(maxit=500)))


#make a summary table for all time frames of simulation data
SimKernelsInt <- as.data.frame(matrix(NA, nrow = 6, ncol = 10))
setDT(SimKernelsInt)
setnames(SimKernelsInt, c("data_source", "time_scale", "time_id", "k", "theta","num_matches", "mdd", "med", "dist90", "log_like"))


BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[1] <- i
SimKernelsInt$data_source[1] <- "simulation"
SimKernelsInt$time_scale[1] <- "annual"
SimKernelsInt$time_id[1] <- "2012"
SimKernelsInt$k[1] <-  BestK2012
SimKernelsInt$theta[1] <- BestTheta2012
SimKernelsInt$num_matches[1] <-sum(biophys_par_data2012$Assignments[1:nrow(biophys_par_data2012$Assignments)-1,])
SimKernelsInt$mdd[1] <- MDD2012
SimKernelsInt$med[1] <- Med2012
SimKernelsInt$dist90[1] <- Dist90_2012
SimKernelsInt$log_like[1] <- logLik(Sim2012Fit)[1]

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[2] <- i
SimKernelsInt$data_source[2] <- "simulation"
SimKernelsInt$time_scale[2] <- "annual"
SimKernelsInt$time_id[2] <- "2013"
SimKernelsInt$k[2] <-  BestK2013
SimKernelsInt$theta[2] <- BestTheta2013
SimKernelsInt$num_matches[2] <-sum(biophys_par_data2013$Assignments[1:nrow(biophys_par_data2013$Assignments)-1,])
SimKernelsInt$mdd[2] <- MDD2013
SimKernelsInt$med[2] <- Med2013
SimKernelsInt$dist90[2] <- Dist90_2013
SimKernelsInt$log_like[2] <- logLik(Sim2013Fit)[1]


BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[3] <- i
SimKernelsInt$data_source[3] <- "simulation"
SimKernelsInt$time_scale[3] <- "annual"
SimKernelsInt$time_id[3] <- "2014"
SimKernelsInt$k[3] <-  BestK2014
SimKernelsInt$theta[3] <- BestTheta2014
SimKernelsInt$num_matches[3] <-sum(biophys_par_data2014$Assignments[1:nrow(biophys_par_data2014$Assignments)-1,])
SimKernelsInt$mdd[3] <- MDD2014
SimKernelsInt$med[3] <- Med2014
SimKernelsInt$dist90[3] <- Dist90_2014
SimKernelsInt$log_like[3] <- logLik(Sim2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Sim2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)
    
SimKernelsInt$iteration[4] <- i
SimKernelsInt$data_source[4] <- "simulation"
SimKernelsInt$time_scale[4] <- "total_years"
SimKernelsInt$time_id[4] <- "2012-4"
SimKernelsInt$k[4] <-  BestK2012_4
SimKernelsInt$theta[4] <- BestTheta2012_4
SimKernelsInt$num_matches[4] <-sum(biophys_par_data2012_4$Assignments[1:nrow(biophys_par_data2012_4$Assignments)-1,])
SimKernelsInt$mdd[4] <- MDD2012_4
SimKernelsInt$med[4] <- Med2012_4
SimKernelsInt$dist90[4] <- Dist90_2012_4
SimKernelsInt$log_like[4] <- logLik(Sim2012_4Fit)[1]

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[5] <- i
SimKernelsInt$data_source[5] <- "simulation"
SimKernelsInt$time_scale[5] <- "monsoonal"
SimKernelsInt$time_id[5] <- "NEM"
SimKernelsInt$k[5] <-  BestKNEM
SimKernelsInt$theta[5] <- BestThetaNEM
SimKernelsInt$num_matches[5] <-sum(biophys_par_dataNEM$Assignments[1:nrow(biophys_par_dataNEM$Assignments)-1,])
SimKernelsInt$mdd[5] <- MDDNEM
SimKernelsInt$med[5] <- MedNEM
SimKernelsInt$dist90[5] <- Dist90_NEM
SimKernelsInt$log_like[5] <- logLik(SimNEMFit)[1]

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[6] <- i
SimKernelsInt$data_source[6] <- "simulation"
SimKernelsInt$time_scale[6] <- "monsoonal"
SimKernelsInt$time_id[6] <- "SWM"
SimKernelsInt$k[6] <-  BestKSWM
SimKernelsInt$theta[6] <- BestThetaSWM
SimKernelsInt$num_matches[6] <-sum(biophys_par_dataSWM$Assignments[1:nrow(biophys_par_dataSWM$Assignments)-1,])
SimKernelsInt$mdd[6] <- MDDSWM
SimKernelsInt$med[6] <- MedSWM
SimKernelsInt$dist90[6] <- Dist90_SWM
SimKernelsInt$log_like[6] <- logLik(SimSWMFit)[1]



#make a summary table for all time frames of genetic data
GenKernelsInt <- as.data.frame(matrix(NA, nrow = 6, ncol = 10))
setDT(GenKernelsInt)
setnames(GenKernelsInt, c("data_source", "time_scale", "time_id", "k", "theta", "num_matches", "mdd", "med", "dist90", "log_like"))
#GenKernelsInt <- data.table(data_source=time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric())


BestK2012 <- as.numeric(coef(Gen2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Gen2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[1] <- i
GenKernelsInt$data_source[1] <- "genetics"    
GenKernelsInt$time_scale[1] <- "annual"
GenKernelsInt$time_id[1] <- "2012"
GenKernelsInt$k[1] <-  BestK2012
GenKernelsInt$theta[1] <- BestTheta2012
GenKernelsInt$num_matches[1] <-sum(par_data2012$Assignments[1:nrow(par_data2012$Assignments)-1,])
GenKernelsInt$mdd[1] <- MDD2012
GenKernelsInt$med[1] <- Med2012
GenKernelsInt$dist90[1] <- Dist90_2012
GenKernelsInt$log_like[1] <- logLik(Gen2012Fit)[1]

BestK2013 <- as.numeric(coef(Gen2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Gen2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[2] <- i
GenKernelsInt$data_source[2] <- "genetics"    
GenKernelsInt$time_scale[2] <- "annual"
GenKernelsInt$time_id[2] <- "2013"
GenKernelsInt$k[2] <-  BestK2013
GenKernelsInt$theta[2] <- BestTheta2013
GenKernelsInt$num_matches[2] <-sum(par_data2013$Assignments[1:nrow(par_data2013$Assignments)-1,])
GenKernelsInt$mdd[2] <- MDD2013
GenKernelsInt$med[2] <- Med2013
GenKernelsInt$dist90[2] <- Dist90_2013
GenKernelsInt$log_like[2] <- logLik(Gen2013Fit)[1]

BestK2014 <- as.numeric(coef(Gen2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Gen2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[3] <- i
GenKernelsInt$data_source[3] <- "genetics"    
GenKernelsInt$time_scale[3] <- "annual"
GenKernelsInt$time_id[3] <- "2014"
GenKernelsInt$k[3] <-  BestK2014
GenKernelsInt$theta[3] <- BestTheta2014
GenKernelsInt$num_matches[3] <-sum(par_data2014$Assignments[1:nrow(par_data2014$Assignments)-1,])
GenKernelsInt$mdd[3] <- MDD2014
GenKernelsInt$med[3] <- Med2014
GenKernelsInt$dist90[3] <- Dist90_2014
GenKernelsInt$log_like[3] <- logLik(Gen2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Gen2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Gen2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[4] <- i
GenKernelsInt$data_source[4] <- "genetics"    
GenKernelsInt$time_scale[4] <- "total_years"
GenKernelsInt$time_id[4] <- "2012-4"
GenKernelsInt$k[4] <-  BestK2012_4
GenKernelsInt$theta[4] <- BestTheta2012_4
GenKernelsInt$num_matches[4] <-sum(par_data2012_4$Assignments[1:nrow(par_data2012_4$Assignments)-1,])
GenKernelsInt$mdd[4] <- MDD2012_4
GenKernelsInt$med[4] <- Med2012_4
GenKernelsInt$dist90[4] <- Dist90_2012_4
GenKernelsInt$log_like[4] <- logLik(Gen2012_4Fit)[1]

BestKNEM <- as.numeric(coef(GenNEMFit)[1])
BestThetaNEM <- as.numeric(coef(GenNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[5] <- i
GenKernelsInt$data_source[5] <- "genetics"    
GenKernelsInt$time_scale[5] <- "monsoonal"
GenKernelsInt$time_id[5] <- "NEM"
GenKernelsInt$k[5] <-  BestKNEM
GenKernelsInt$theta[5] <- BestThetaNEM
GenKernelsInt$num_matches[5] <-sum(par_dataNEM$Assignments[1:nrow(par_dataNEM$Assignments)-1,])
GenKernelsInt$mdd[5] <- MDDNEM
GenKernelsInt$med[5] <- MedNEM
GenKernelsInt$dist90[5] <- Dist90_NEM
GenKernelsInt$log_like[5] <- logLik(GenNEMFit)[1]

BestKSWM <- as.numeric(coef(GenSWMFit)[1])
BestThetaSWM <- as.numeric(coef(GenSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[6] <- i
GenKernelsInt$data_source[6] <- "genetics"    
GenKernelsInt$time_scale[6] <- "monsoonal"
GenKernelsInt$time_id[6] <- "SWM"
GenKernelsInt$k[6] <-  BestKSWM
GenKernelsInt$theta[6] <- BestThetaSWM
GenKernelsInt$num_matches[6] <-sum(par_dataSWM$Assignments[1:nrow(par_dataSWM$Assignments)-1,])
GenKernelsInt$mdd[6] <- MDDSWM
GenKernelsInt$med[6] <- MedSWM
GenKernelsInt$dist90[6] <- Dist90_SWM
GenKernelsInt$log_like[6] <- logLik(GenSWMFit)[1]

GenKernelsInt$iteration <- i
GenKernelsInt$data_source <- "genetics"

        BootKernelsInt <- rbind(SimKernelsInt, GenKernelsInt)
   
        BootKernels <- rbind(BootKernels, BootKernelsInt)
    
 #also bootstrap the biophysical likelihood
Data_par2012_bio2012 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2012[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012[1:nrow(genetic_parentage_matrices$GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2012,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012[nrow(genetic_parentage_matrices$GenMat2012),]))

Data_par2013_bio2013 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2013[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2013[1:nrow(genetic_parentage_matrices$GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2013,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2013[nrow(genetic_parentage_matrices$GenMat2013),]))

Data_par2014_bio2014 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2014[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2014[1:nrow(genetic_parentage_matrices$GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2014[nrow(genetic_parentage_matrices$GenMat2014),]))


#total compared to total
Data_par2012_4_bio2012_4 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012_4[1:nrow(genetic_parentage_matrices$GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012_4[nrow(genetic_parentage_matrices$GenMat2012_4),]))

#for seasonal kernels

 Data_parNEM_bioNEM <- list(BioPhysMat=as.matrix(biophys_norm_matrices$MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(genetic_parentage_matrices$GenMatNEM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),]))

 Data_parSWM_bioSWM <- list(BioPhysMat=as.matrix(biophys_norm_matrices$MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(genetic_parentage_matrices$GenMatSWM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))
#total to total 
 Data_parNEMSWM_bio2012_4 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=(as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(genetic_parentage_matrices$GenMatNEM)-1,])+
                                  as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(genetic_parentage_matrices$GenMatSWM)-1,])), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),])+
                    as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))

#calculate likelihood
LL_annual_annual <- LL_biophys(Data_par2012_bio2012)+LL_biophys(Data_par2013_bio2013)+LL_biophys(Data_par2014_bio2014)
LL_total_total <- LL_biophys(Data_par2012_4_bio2012_4)
LL_monsoonal_monsoonal <- LL_biophys(Data_parNEM_bioNEM)+LL_biophys(Data_parSWM_bioSWM)
LL_monsoonal_total <- LL_biophys(Data_parNEMSWM_bio2012_4)

#store results
LLInt <- data.table(comparison= c("gen_total_bio_total", "gen_annual_bio_annual", "gen_monsoonal_bio_monsoonal", "gen_monsoonal_bio_total"), 
           time_scale= c("total_years", "annual", "total_monsoons", "monsoonal"),
           log_like=c(LL_total_total, LL_annual_annual, LL_monsoonal_monsoonal, LL_monsoonal_total))

LLInt$iteration <- i
    
BootLikelihood <- rbind(BootLikelihood, LLInt)
    
    setTxtProgressBar(pb, i)
    }
close(pb)

fwrite(BootKernels, "~/oceanography/script_output/KernelFits/summary_tables/BootstrappedBiophysicalKernels08DayPLD.csv")
fwrite(BootLikelihood, "~/oceanography/script_output/KernelFits/summary_tables/BootstrappedBiophysicalLikelihoods08DayPLD.csv")

  |======================================================================| 100%


In [269]:
(BootKernels)

data_source,time_scale,time_id,k,theta,num_matches,mdd,med,dist90,log_like,iteration
simulation,annual,2012,-1.9185021,1.4040589,165651,4.718663e+00,3.6642,10.2717,-4.188598e+05,1
simulation,annual,2013,-1.8717581,1.3624508,144781,4.614830e+00,3.5528,10.0951,-1.224392e+06,1
simulation,annual,2014,-1.6718280,1.2167054,152667,4.195828e+00,3.1197,9.3515,-1.101513e+06,1
simulation,total_years,2012-4,-3.0000000,1.0000000,463565,2.008554e+01,13.9222,46.2487,1.600000e+13,1
simulation,monsoonal,NEM,-1.7203774,1.2498612,440921,4.288058e+00,3.2158,9.5145,-3.053109e+06,1
simulation,monsoonal,SWM,-1.9875929,1.4726017,22644,4.876493e+00,3.8370,10.5338,-5.362466e+05,1
genetics,annual,2012,-3.2327756,3.4792355,3,1.255470e+01,11.5660,24.2084,-1.736390e+01,1
genetics,annual,2013,-3.2596633,5.0000000,18,1.258213e+01,11.9962,23.5500,-9.821552e+01,1
genetics,annual,2014,2.1372418,0.1500000,13,1.982534e+06,26148.7047,591064.4592,-2.006598e+02,1
genetics,total_years,2012-4,-2.4009249,2.1756092,71,6.033175e+00,5.1839,12.2920,-1.745478e+03,1


In [ ]:
par_data2012 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2012 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012, control=list(maxit=500)))

par_data2013 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2013 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2013, control=list(maxit=500)))

par_data2014 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2014, control=list(maxit=500)))

par_data2012_4 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2012_4, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012_4, control=list(maxit=500)))

par_dataNEM <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2014  & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014   & prop_anem_samp >0, .(prop_anem_samp)]))
GenNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataNEM, control=list(maxit=500)))

par_dataSWM <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyDataNSMag[year==2014  & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyDataNSMag[year==2014   & prop_anem_samp >0, .(prop_anem_samp)]))
GenSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataSWM, control=list(maxit=500)))

